# TP1 - Introdução a Banco de Dados - 2019/2
**Lucas Fonseca Mundim - 2015042134**

---

In [2]:
import pandas as pd
import io
import sqlite3 as sql
import os

In [3]:
connection = sql.connect('tp1.db')

### Executar apenas na primeira iteração:

In [ ]:
cursor = connection.cursor()
data = io.open('despesas_publicas_tp1.sql', 'r', encoding='ISO-8859-1')
database = data.read()
cursor.executescript(database)

In [4]:
def query(query_string):
    return pd.read_sql_query(query_string, connection)

Matrícula: 2015**042134**

Exercícios: 1,5,3,2,4,6 <- Reordenados para 1,2,3,4,5,6
#### Questão 1
Liste todos os códigos e nomes de funções da subfunção que possui o maior número de funções.

In [5]:
# Recover the sub-function with most functions
first = query('select cod_funcao, cod_subfuncao, count(cod_subfuncao) as count from detalhamento_funcao group by cod_subfuncao limit 1')
# Recover the functions belonging to the found sub-function
second = query('select cod_funcao from detalhamento_funcao where cod_subfuncao=122')
# Recover the final code and name for the wanted functions
third = query('select * from funcao where cod_funcao in (5,6,26)')

print(first)
print(second)
print(third)

# All at once...
q1_last = ('select funcao.nome_funcao, funcao.cod_funcao, detalhamento_funcao.cod_subfuncao ' + 
'from funcao ' +
'natural join detalhamento_funcao ' +
'where detalhamento_funcao.cod_subfuncao=(' +
'select cod_subfuncao from detalhamento_funcao group by cod_subfuncao limit 1)')
    
q1 = query(q1_last)
q1

   COD_FUNCAO  COD_SUBFUNCAO  count
0           5            122      3
   COD_FUNCAO
0           5
1           6
2          26
   COD_FUNCAO        NOME_FUNCAO
0           5    Defesa nacional
1           6  Segurança pública
2          26         Transporte


,NOME_FUNCAO,COD_FUNCAO,COD_SUBFUNCAO
0,Defesa nacional,5,122
1,Segurança pública,6,122
2,Transporte,26,122


#### Questão 2
Liste a média de despesas públicas por função (nome) e subfunção (nome), apenas nos casos em que a média excedeu mil reais.

In [6]:
q2_query = ('select * from('
    'select ' +
    'funcao.nome_funcao, ' + 
    'subfuncao.nome_subfuncao, ' +
    'avg(valor_despesa) as media ' +
    'from despesa_publica ' +
    'natural join detalhamento_funcao ' +
    'natural join funcao ' +
    'natural join subfuncao ' +
    'group by nome_funcao, nome_subfuncao ' +
    'order by media desc ' +
    ') where media >= 1000')
q2 = query(q2_query)
q2

,nome_funcao,nome_subfuncao,media
0,Educação,Educação básica,4398.185000
1,Assistência social,Tecnologia da informação,2266.087692
2,Educação,Ensino profissional,1826.135152
3,Assistência social,Assistência comunitária,1713.411805


#### Questão 3
Liste todos os nomes dos programas de governo e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [7]:
q2_query = ('select ' +
    'nome_programa_governo, sum(valor_despesa) as despesas_totais ' +
    'from programa_governo ' +
    'natural join (select * from despesa_publica where ano_despesa=2018) ' +
    'group by cod_programa_governo order by despesas_totais desc')
q2 = query(q2_query)
q2

,NOME_PROGRAMA_GOVERNO,despesas_totais
0,BRASIL SEM MISERIA,387175.60
1,ENEM,334262.06
2,PRONATEC,60262.46
3,CONTROLE DO CANCER,38971.19
4,MAIS MEDICOS,14400.00
5,REGULARIZACAO FUNDIARIA,1497.26
6,REDE CEGONHA,1210.00
7,VIVER SEM LIMITE,0.00
8,SOS EMERGENCIA,0.00
9,MELHOR EM CASA,0.00


#### Questão 4
Liste o mês e ano, modalidade, programa de governo e valor de todas as despesas do órgão Ministério da Educação no primeiro semestre de 2018.

In [8]:
q4_query = ('select ' +
    'mes_despesa, ano_despesa, programa_governo.nome_programa_governo, valor_despesa ' +
    'from despesa_publica ' + 
    'natural join orgao ' + 
    'natural join programa_governo ' + 
    'where nome_orgao=\'Ministério da Educação\' ' +
    'and ano_despesa=2018 ' +
    'and mes_despesa in (1,2,3,4,5,6)')
q4 = query(q4_query)
q4

,MES_DESPESA,ANO_DESPESA,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,1,2018,VIVER SEM LIMITE,0.0
1,2,2018,VIVER SEM LIMITE,0.0
2,3,2018,VIVER SEM LIMITE,0.0
3,4,2018,VIVER SEM LIMITE,0.0
4,5,2018,VIVER SEM LIMITE,0.0
...,...,...,...,...
148,2,2018,MAIS MEDICOS,0.0
149,3,2018,MAIS MEDICOS,0.0
150,4,2018,MAIS MEDICOS,0.0
151,5,2018,MAIS MEDICOS,0.0


#### Questão 5
Liste o código e nome dos órgãos que possuíram os 5 maiores valores de despesas totais nas funções de educação e saúde durante o ano de 2018.

In [9]:
q5_query = (
    'select cod_orgao, orgao.nome_orgao, sum(valor_despesa) as despesas_totais ' +
    'from despesa_publica ' +
    'natural join detalhamento_funcao ' +
    'natural join funcao ' +
    'natural join orgao ' +
    'where funcao.nome_funcao in (\'Saúde\', \'Educação\') ' +
    'group by cod_orgao order by despesas_totais desc'
)

q5 = query(q5_query)
q5

,COD_ORGAO,NOME_ORGAO,despesas_totais
0,52000,Ministério da Defesa,334126.70
1,26000,Ministério da Educação,64007.82
2,36000,Ministério da Saúde,50971.19
3,24000,"Ministério da Ciência, Tecnologia, Inovações",0.00
4,30000,Ministério da Justiça e Segurança Pública,0.00


#### Questão 6
Liste todos os nomes de funções e subfunções das despesas públicas dos últimos 5 meses de 2018 e seus respectivos valores totais, ordenando de forma decrescente pelo valor.

In [16]:
q6_query = (
    'select * from('
    'select nome_funcao, nome_subfuncao, sum(valor_despesa) as despesas_totais ' +
    'from despesa_publica ' +
    'natural join detalhamento_funcao ' +
    'natural join funcao ' +
    'natural join subfuncao ' +
    'where despesa_publica.ano_despesa = 2018 '
    'and despesa_publica.mes_despesa >= 8 '
    'group by nome_subfuncao, nome_funcao order by despesas_totais desc)'
    'where despesas_totais > 0'
)

q6 = query(q6_query)
q6

,nome_funcao,nome_subfuncao,despesas_totais
0,Educação,Educação básica,334126.70
1,Assistência social,Alimentação e nutrição,100373.55
2,Educação,Ensino profissional,7011.38
3,Educação,Ensino superior,2400.00
4,Saúde,Atenção básica,1000.00
